In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import random
import warnings
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
from sklearn.exceptions import ConvergenceWarning

try:
    from myst_nb import glue
except ImportError:
    glue = lambda *args, **kwargs: _
# 一部の警告を無視
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.simplefilter("ignore", FutureWarning)
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
Xs=[]
Ys=[]
def load_data(data_name,N,M,K,pixel):
    for id in range(K):
        # id=random.randint(0,K-1)
        img=cv2.imread(f'./training_data/{data_name}/numbers_{id}.png')
        numbers=np.load(f'./training_data/{data_name}/numbers_{id}.npy')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # plt.imshow(img, cmap='gray')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # plt.imshow(img, cmap='gray')
        # print(img.shape)
        img_rs=cv2.resize(gray,(pixel*M,pixel*N),interpolation=cv2.INTER_AREA)
    # img_rs=cv2.bitwise_not(img_rs)
    # print(img_rs.shape,i)

        for i in range(N):
            for j in range(M):
                digit_square = img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel]
                x_i=digit_square.reshape(1, -1)
                # print(x_i.shape,i,j)
                y=numbers[i,j]
                Xs.append(x_i)
                Ys.append(y)

In [3]:
# 白地のまま学習する

# 画像を読み込む
try:
    train
except:
    train=1
save=1
pixel=20
N,M=50,50
K=2
# data_name='numbers'
# load_data(data_name,N,M,K,pixel)
# load_data('numbers',50,50,10,pixel)
load_data('numbers_v2',10,10,500,pixel)
load_data('numbers_line_3',10,10,500,pixel)
# load_data('numbers_line',50,50,10,pixel)
load_data('numbers_ln_v2',10,10,500,pixel)
load_data('numbers_ln_m',10,10,500,pixel)
# load_data('numbers_line_wb',50,50,10,pixel)
# load_data('numbers_line_wb_light',10,10,100,pixel)
# load_data('numbers_wo0',50,50,10,pixel)
# load_data('numbers_wo0_line',50,50,10,pixel)


        
Xs=pd.DataFrame(np.array(Xs).reshape(-1,pixel*pixel))
Ys=pd.DataFrame(np.array(Ys).reshape(-1,1),columns=['label'])
print(Xs.shape,Ys.shape)
df=pd.concat([Ys,Xs],axis=1)
# df=df.loc[df.iloc[:,0]!=0]
df.shape


(200000, 400) (200000, 1)


(200000, 401)

In [4]:
# i,j=10,5
# plt.imshow(img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel], cmap='gray')


In [5]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
X_org, y_org = df.iloc[:,1:], df.iloc[:,0]
X_org = np.array(X_org / 255.0, dtype="double")
y_org = np.array(y_org, dtype="uint8")
X, X_test, y, y_test = model_selection.train_test_split(
    X_org, y_org,test_size=0.2, shuffle=True,random_state=42
)
# use partial data 
# n_samples = 10000  
# X, y = X[:n_samples], y[:n_samples]
# X_test, y_test = X_test[:n_samples], y_test[:n_samples]

# スケーリングパラメータの計算
scaler = StandardScaler()
scaler.fit(X)

# データのスケーリング
X = scaler.transform(X)
X_test = scaler.transform(X_test)


In [6]:
model_name="ensemble_MLPC"
# model_name="MLPC_numbers_mix"
clf=pd.read_pickle(f"./pickle/{model_name}_clf.pickle")

acc_train = 100.0 * clf.score(X, y)
print("logis_acc_train", acc_train)

# テストデータに対する識別精度の計算
acc_test = 100.0 * clf.score(X_test, y_test)
print("logis_acc_test", acc_test)
# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
labels=[0,1,2,3,4,5,6,7,8,9]
cm = confusion_matrix(y_test, y_pred,labels=labels)
print(cm)

logis_acc_train 95.768125
logis_acc_test 95.77
[[3991    1    0    2    0    2    1    1    0    1]
 [  13 3966   27   52   33    8    2   15    1    2]
 [   4   10 3891   47    9   16    8   82    9   50]
 [   4   19   84 3884    6   35    2    9   10   31]
 [   2   21   27    7 3852    7   31    2   10   14]
 [   8   22   11   45    8 3642  109    1   55   31]
 [   5    5   19    3   23   25 3900    1   70    8]
 [   2   19   26    4    2   18    2 3738    0   42]
 [   0    2   30   12   16   63   30    2 3689   48]
 [   8    4   14   18   14  100   26    7   17 3755]]


In [7]:
print("Confusion Matrix:")
print("Labels: ", labels)
print("      ", end="")
for label in labels:
    print(f"{label:4d}", end=" ")
print()
for i, row in enumerate(cm):
    print(f"    {i:2d} ", end="")
    for val in row:
        print(f"{val:4d}", end=" ")
    print()

Confusion Matrix:
Labels:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
         0    1    2    3    4    5    6    7    8    9 
     0 3991    1    0    2    0    2    1    1    0    1 
     1   13 3966   27   52   33    8    2   15    1    2 
     2    4   10 3891   47    9   16    8   82    9   50 
     3    4   19   84 3884    6   35    2    9   10   31 
     4    2   21   27    7 3852    7   31    2   10   14 
     5    8   22   11   45    8 3642  109    1   55   31 
     6    5    5   19    3   23   25 3900    1   70    8 
     7    2   19   26    4    2   18    2 3738    0   42 
     8    0    2   30   12   16   63   30    2 3689   48 
     9    8    4   14   18   14  100   26    7   17 3755 


In [8]:
results = []
for i in range(len(labels)):
    col = [row[i] for row in cm]
    top2 = np.argsort(col)[::-1][:3]
    results.append([labels[top2[0]], labels[top2[1]], labels[top2[2]]])

# 結果の出力
for i, col in enumerate(results):
    print(f"{labels[i]}: {col}")

0: [0, 1, 9]
1: [1, 5, 4]
2: [2, 3, 8]
3: [3, 1, 2]
4: [4, 1, 6]
5: [5, 9, 8]
6: [6, 5, 4]
7: [7, 2, 1]
8: [8, 6, 5]
9: [9, 2, 8]


In [9]:
results

[[0, 1, 9],
 [1, 5, 4],
 [2, 3, 8],
 [3, 1, 2],
 [4, 1, 6],
 [5, 9, 8],
 [6, 5, 4],
 [7, 2, 1],
 [8, 6, 5],
 [9, 2, 8]]

In [10]:
for i in range(3**5):
    nums = [(i // 3**j) % 3 for j in range(5)]
    print(nums)

[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[2, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 1, 0, 0, 0]
[2, 1, 0, 0, 0]
[0, 2, 0, 0, 0]
[1, 2, 0, 0, 0]
[2, 2, 0, 0, 0]
[0, 0, 1, 0, 0]
[1, 0, 1, 0, 0]
[2, 0, 1, 0, 0]
[0, 1, 1, 0, 0]
[1, 1, 1, 0, 0]
[2, 1, 1, 0, 0]
[0, 2, 1, 0, 0]
[1, 2, 1, 0, 0]
[2, 2, 1, 0, 0]
[0, 0, 2, 0, 0]
[1, 0, 2, 0, 0]
[2, 0, 2, 0, 0]
[0, 1, 2, 0, 0]
[1, 1, 2, 0, 0]
[2, 1, 2, 0, 0]
[0, 2, 2, 0, 0]
[1, 2, 2, 0, 0]
[2, 2, 2, 0, 0]
[0, 0, 0, 1, 0]
[1, 0, 0, 1, 0]
[2, 0, 0, 1, 0]
[0, 1, 0, 1, 0]
[1, 1, 0, 1, 0]
[2, 1, 0, 1, 0]
[0, 2, 0, 1, 0]
[1, 2, 0, 1, 0]
[2, 2, 0, 1, 0]
[0, 0, 1, 1, 0]
[1, 0, 1, 1, 0]
[2, 0, 1, 1, 0]
[0, 1, 1, 1, 0]
[1, 1, 1, 1, 0]
[2, 1, 1, 1, 0]
[0, 2, 1, 1, 0]
[1, 2, 1, 1, 0]
[2, 2, 1, 1, 0]
[0, 0, 2, 1, 0]
[1, 0, 2, 1, 0]
[2, 0, 2, 1, 0]
[0, 1, 2, 1, 0]
[1, 1, 2, 1, 0]
[2, 1, 2, 1, 0]
[0, 2, 2, 1, 0]
[1, 2, 2, 1, 0]
[2, 2, 2, 1, 0]
[0, 0, 0, 2, 0]
[1, 0, 0, 2, 0]
[2, 0, 0, 2, 0]
[0, 1, 0, 2, 0]
[1, 1, 0, 2, 0]
[2, 1, 0, 2, 0]
[0, 2, 0, 2, 0]
[1, 2, 0, 2, 0]
[2, 2, 0